In [1]:
%matplotlib inline

import geopandas as gpd

import pandas as pd
import os
import requests
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
import matplotlib.font_manager as fm

for font in fm.fontManager.ttflist:
    if font.name in ['AppleGothic', 'Malgun Gothic']:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['font.family']

['Malgun Gothic']

In [3]:
#2018년 7월 승차일 기준 1~4일의 버스 카드태깅 정보를 담고있습니다.
#미터 / 분
TripChain = pd.read_csv('../Data/PJT001_TripChain.csv')

#17~18년 기준, 경기도 버스 정류장에 대한 정보를 담고있습니다.
# 성보님과 은솔님이 채워주신 데이터 바로 이용
StationTable = pd.read_csv('../Data/station_null.csv')

#2018년 7월 1~4일의 행정동별 이동 인구수 정보를 담고 있습니다.
AreaPeople = pd.read_csv('../Data/PJT001_sk_emd_od.csv')

#버스노선-정류장 매핑 테이블 정보 입니다.
RouteStationInfo = pd.read_csv('../Data/PJT001_routestationinfo.csv')

#
RouteStationMapping = pd.read_csv('../Data/PJT001_routestationmapping.csv')

In [4]:
RouteStation_latlon = pd.read_csv('df_route_stations.csv', encoding='euc-kr')

In [5]:
TripChain.columns=['암호화카드번호', '트랜잭션ID', '환승횟수', '교통카드발행사ID',
       '총이용객수', '사용자구분', '교통수단CD1', '교통수단CD2',
       '교통수단CD3', '교통수단CD4', '교통수단CD5', '버스노선ID1',
       '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
       '차량ID1', '차량ID2', '차량ID3', '차량ID4',
       '차량ID5', '총통행거리', '총탑승시간', '총소요시간',
       '승차일시1', '승차일시2', '승차일시3', '승차일시4',
       '승차일시5', '하차일시1', '하차일시2', '하차일시3',
       '하차일시4', '하차일시5', '최초승차일시', '최종하차일시',
       '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
       '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3',
       '하차역ID4', '하차역ID5', '최초승차역ID', '최종하차역ID',
       '총이용금액', '수집건수', '트립체인완료코드']

In [8]:
TripChain[TripChain['환승횟수']==2].iloc[0]

암호화카드번호         900079696430
트랜잭션ID                    56
환승횟수                       2
교통카드발행사ID            9000923
총이용객수                      1
사용자구분                      1
교통수단CD1                  500
교통수단CD2                  500
교통수단CD3                  NaN
교통수단CD4                  NaN
교통수단CD5                  NaN
버스노선ID1           4.1002e+07
버스노선ID2           4.1002e+07
버스노선ID3                  NaN
버스노선ID4                  NaN
버스노선ID5                  NaN
차량ID1            1.41772e+08
차량ID2            1.41772e+08
차량ID3                    NaN
차량ID4                    NaN
차량ID5                    NaN
총통행거리                  11170
총탑승시간                     25
총소요시간                     25
승차일시1            2.01807e+13
승차일시2            2.01807e+13
승차일시3                    NaN
승차일시4                    NaN
승차일시5                    NaN
하차일시1            2.01807e+13
하차일시2            2.01807e+13
하차일시3                    NaN
하차일시4                    NaN
하차일시5                    NaN
최초승차일시        

In [6]:
# a = RouteStation_latlon['bus_line_no'].unique()
# print('27' in a)

RouteStation_latlon[RouteStation_latlon['bus_line_no'] == '27']

,Unnamed: 0,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no,govern,lat,lng,이비노선ID,표준노선ID
2187,9727,91274,233000096,27,1,조암터미널,233000929,36011.0,화성시,37.082150,126.815583,233000096.0,41008157.0
2188,9728,91275,233000096,27,2,조암삼거리(경유),277102392,NaN,화성시,37.080933,126.814550,233000096.0,41008157.0
2189,9729,91276,233000096,27,3,장안여자중학교,233000992,36797.0,화성시,37.078567,126.815217,233000096.0,41008157.0
2190,9730,91277,233000096,27,4,사곡6리,233000496,36910.0,화성시,37.074817,126.815150,233000096.0,41008157.0
2191,9731,91278,233000096,27,5,장안우체국,233002511,55388.0,화성시,37.072167,126.813850,233000096.0,41008157.0
2192,9732,91279,233000096,27,6,사곡1리,233000993,37478.0,화성시,37.069083,126.809217,233000096.0,41008157.0
2193,9733,91280,233000096,27,7,장자터,233001776,37834.0,화성시,37.066483,126.802167,233000096.0,41008157.0
2194,9734,91281,233000096,27,8,기아생활관,233000485,36918.0,화성시,37.064383,126.795617,233000096.0,41008157.0
2195,9735,91282,233000096,27,9,화산삼거리,233001554,36701.0,화성시,37.062667,126.792467,233000096.0,41008157.0
2196,9736,91283,233000096,27,10,기아2차아파트앞,233002268,55143.0,화성시,37.060467,126.792900,233000096.0,41008157.0


In [7]:
TripChain['최초승차역ID'] = TripChain['승차역ID1']

In [8]:
TripChain.drop_duplicates(subset =["암호화카드번호", "최초승차일시", "최종하차일시"], 
                     keep = 'first', inplace = True)

In [9]:
All_bus_id = RouteStation_latlon['pr_station_id'].unique()
# 화성시 bus id만 저장

In [10]:
All_bus_id = list(All_bus_id)
All_bus_id

[228000204,
 233000004,
 233000005,
 233000010,
 233000013,
 233000015,
 233000019,
 233000021,
 233000022,
 233000024,
 233000025,
 233000026,
 233000027,
 233000028,
 233000029,
 233000030,
 233000035,
 233000034,
 233000036,
 233000038,
 233000040,
 233000041,
 233000043,
 233000044,
 233000045,
 233000046,
 233000047,
 233000048,
 233000049,
 233000051,
 233000052,
 233000056,
 233000057,
 233000058,
 233000059,
 233000060,
 233000062,
 233000064,
 233000067,
 233000075,
 233000077,
 233000079,
 233000080,
 233000081,
 233000084,
 233000085,
 233000086,
 233000094,
 233000095,
 233000096,
 233000117,
 233000125,
 233000126,
 233000128,
 233000130,
 233000131,
 233000133,
 233000134,
 233000135,
 233000136,
 233000139,
 233000140,
 233000141,
 233000142,
 233000144,
 233000252,
 233000253,
 233000254,
 233000258,
 233000263,
 233000264,
 233000265,
 233000266,
 233000267,
 233000268,
 233000269,
 233000270,
 233000271,
 233000272,
 233000273,
 233000274,
 233000275,
 233000276,
 233

In [11]:
len(All_bus_id)

236

In [12]:
RouteStation_latlon = RouteStation_latlon.sort_values('pr_station_id', ascending=True)
RouteStation_latlon.shape

(10580, 13)

In [13]:
RouteStationMapping.shape

(2127, 6)

In [14]:
RouteStationMapping.sample()

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명
815,경기시내,경원여객,4100700,216000023,41007032,88


In [15]:
RouteStationMapping['isHwasung'] = RouteStationMapping['이비노선ID'].apply(lambda x: True if x in All_bus_id else False)
RouteStationMapping.head()

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung
0,경기시내,경원여객M,2805000,216000044,28050900,M6410,False
1,경기시내,강화운수,4100100,232000028,41001001,2,False
2,경기시내,강화운수,4100100,232000029,41001013,88,False
3,경기시내,강화운수,4100100,232000061,41001020,3000,False
4,경기시내,강화운수,4100100,232000067,41001024,388,False


In [16]:
Route_Mapping = RouteStationMapping[RouteStationMapping['isHwasung'] == True]
Route_Mapping.head()

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung
53,경기시내,경기고속,4100200,234000068,41002090,720-3,True
759,경기시내,경남여객,4100600,228000204,41006281,24,True
862,경기시내,경진여객,4100800,233000010,41008001,33-1,True
863,경기시내,경진여객,4100800,233000013,41008101,2,True
864,경기시내,경진여객,4100800,233000015,41008103,4,True


In [17]:
Route_Mapping.shape

(78, 7)

In [18]:
Route_Mapping['p_1'] = 0
Route_Mapping['p_2'] = 0
Route_Mapping['p_4'] = 0
Route_Mapping['p_8'] = 0
Route_Mapping.drop_duplicates(subset =["이비노선ID", "표준노선ID"], 
                     keep = 'first', inplace = True)
Route_Mapping.head()

C:\Users\Leila\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Leila\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Leila\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung,p_1,p_2,p_4,p_8
53,경기시내,경기고속,4100200,234000068,41002090,720-3,True,0,0,0,0
759,경기시내,경남여객,4100600,228000204,41006281,24,True,0,0,0,0
862,경기시내,경진여객,4100800,233000010,41008001,33-1,True,0,0,0,0
863,경기시내,경진여객,4100800,233000013,41008101,2,True,0,0,0,0
864,경기시내,경진여객,4100800,233000015,41008103,4,True,0,0,0,0


In [19]:
normal_id_h = Route_Mapping['표준노선ID'].unique()
normal_id_h

array([41002090, 41006281, 41008001, 41008101, 41008103, 41008110,
       41008112, 41008115, 41008114, 41008105, 41008117, 41008108,
       41008102, 41008107, 41008106, 41008118, 41008119, 41008120,
       41008122, 41008124, 41008125, 41008126, 41008127, 41008128,
       41008129, 41008130, 41008131, 41008132, 41008133, 41008134,
       41008138, 41008139, 41008140, 41008141, 41008142, 41008144,
       41008137, 41008145, 41008146, 41008147, 41008150, 41008151,
       41008155, 41008156, 41008157, 41008161, 41008167, 41008171,
       41037102, 41037001, 41037109, 41037106, 41037113, 41037114,
       41061018, 41061020, 41061022, 41061024, 41061027, 41075031,
       41075017, 41075030, 41075034, 41075035, 41086001, 41086002,
       41086005, 41086004, 41086009, 41086010, 41086012, 41086013,
       41086014, 41086015, 41003015, 41086008, 41086011], dtype=int64)

In [20]:
RouteStationMapping[RouteStationMapping['노선명']=='27']

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung
397,경기시내,대원고속,4100300,234000538,41003438,27,False
908,경기시내,경진여객,4100800,233000096,41008157,27,True
1222,경기시내,명진여객,4101700,207000091,41017113,27,False
1474,경기시내,수원여객,4103100,200000074,41031052,27,False
1808,경기시내,협진여객,4104400,214000046,41044109,27,False
1823,경기시내,화영운수,4104500,213000003,41045008,27,False
1871,경기시내,성광운수,4105600,210000063,41056002,27,False


일반인(1)의 버스노선ID1

In [21]:
new_TripChain_1 = TripChain[TripChain['사용자구분']==1]
new_TripChain_1.shape

(943298, 51)

In [22]:
p_1_id1 = new_TripChain_1['버스노선ID1']

In [23]:
for i in p_1_id1:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_1'] += 1
Route_Mapping

C:\Users\Leila\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung,p_1,p_2,p_4,p_8
53,경기시내,경기고속,4100200,234000068,41002090,720-3,True,13093,0,0,0
759,경기시내,경남여객,4100600,228000204,41006281,24,True,129,0,0,0
862,경기시내,경진여객,4100800,233000010,41008001,33-1,True,83,0,0,0
863,경기시내,경진여객,4100800,233000013,41008101,2,True,2,0,0,0
864,경기시내,경진여객,4100800,233000015,41008103,4,True,4,0,0,0
865,경기시내,경진여객,4100800,233000019,41008110,5-2,True,2,0,0,0
866,경기시내,경진여객,4100800,233000021,41008112,5-4,True,1,0,0,0
867,경기시내,경진여객,4100800,233000022,41008115,15,True,0,0,0,0
868,경기시내,경진여객,4100800,233000024,41008114,14,True,0,0,0,0
869,경기시내,경진여객,4100800,233000025,41008105,9,True,3,0,0,0


일반인(1)의 버스노선ID2

In [25]:
p_1_id2 = new_TripChain_1['버스노선ID2']
for i in p_1_id2:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_1'] += 1
Route_Mapping

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung,p_1,p_2,p_4,p_8
53,경기시내,경기고속,4100200,234000068,41002090,720-3,True,19531,0,0,0
759,경기시내,경남여객,4100600,228000204,41006281,24,True,335,0,0,0
862,경기시내,경진여객,4100800,233000010,41008001,33-1,True,231,0,0,0
863,경기시내,경진여객,4100800,233000013,41008101,2,True,2,0,0,0
864,경기시내,경진여객,4100800,233000015,41008103,4,True,4,0,0,0
865,경기시내,경진여객,4100800,233000019,41008110,5-2,True,2,0,0,0
866,경기시내,경진여객,4100800,233000021,41008112,5-4,True,1,0,0,0
867,경기시내,경진여객,4100800,233000022,41008115,15,True,0,0,0,0
868,경기시내,경진여객,4100800,233000024,41008114,14,True,0,0,0,0
869,경기시내,경진여객,4100800,233000025,41008105,9,True,3,0,0,0


일반인(1)의 버스노선ID3~5

In [26]:
p_1_id3 = new_TripChain_1['버스노선ID3']
for i in p_1_id3:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_1'] += 1

p_1_id4 = new_TripChain_1['버스노선ID4']
for i in p_1_id4:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_1'] += 1

p_1_id5 = new_TripChain_1['버스노선ID5']
for i in p_1_id5:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_1'] += 1
Route_Mapping

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung,p_1,p_2,p_4,p_8
53,경기시내,경기고속,4100200,234000068,41002090,720-3,True,20430,0,0,0
759,경기시내,경남여객,4100600,228000204,41006281,24,True,369,0,0,0
862,경기시내,경진여객,4100800,233000010,41008001,33-1,True,277,0,0,0
863,경기시내,경진여객,4100800,233000013,41008101,2,True,4,0,0,0
864,경기시내,경진여객,4100800,233000015,41008103,4,True,7,0,0,0
865,경기시내,경진여객,4100800,233000019,41008110,5-2,True,2,0,0,0
866,경기시내,경진여객,4100800,233000021,41008112,5-4,True,1,0,0,0
867,경기시내,경진여객,4100800,233000022,41008115,15,True,0,0,0,0
868,경기시내,경진여객,4100800,233000024,41008114,14,True,0,0,0,0
869,경기시내,경진여객,4100800,233000025,41008105,9,True,3,0,0,0


어린이(2) 의 버스노선ID1~5

In [27]:
new_TripChain_2 = TripChain[TripChain['사용자구분']==2]
new_TripChain_2.shape

(6292, 51)

In [28]:
p_2_id1 = new_TripChain_2['버스노선ID1']
for i in p_2_id1:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_2'] += 1

p_2_id2 = new_TripChain_2['버스노선ID2']
for i in p_2_id2:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_2'] += 1

p_2_id3 = new_TripChain_2['버스노선ID3']
for i in p_2_id3:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_2'] += 1

p_2_id4 = new_TripChain_2['버스노선ID4']
for i in p_2_id4:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_2'] += 1

p_2_id5 = new_TripChain_2['버스노선ID5']
for i in p_2_id5:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_2'] += 1

In [29]:
Route_Mapping

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung,p_1,p_2,p_4,p_8
53,경기시내,경기고속,4100200,234000068,41002090,720-3,True,20430,68,0,0
759,경기시내,경남여객,4100600,228000204,41006281,24,True,369,0,0,0
862,경기시내,경진여객,4100800,233000010,41008001,33-1,True,277,0,0,0
863,경기시내,경진여객,4100800,233000013,41008101,2,True,4,0,0,0
864,경기시내,경진여객,4100800,233000015,41008103,4,True,7,0,0,0
865,경기시내,경진여객,4100800,233000019,41008110,5-2,True,2,0,0,0
866,경기시내,경진여객,4100800,233000021,41008112,5-4,True,1,0,0,0
867,경기시내,경진여객,4100800,233000022,41008115,15,True,0,0,0,0
868,경기시내,경진여객,4100800,233000024,41008114,14,True,0,0,0,0
869,경기시내,경진여객,4100800,233000025,41008105,9,True,3,0,0,0


청소년(4)의 버스노선ID1~5

In [30]:
new_TripChain_4 = TripChain[TripChain['사용자구분']==4]
new_TripChain_4.shape

(96370, 51)

In [31]:
p_4_id1 = new_TripChain_4['버스노선ID1']
for i in p_4_id1:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_4'] += 1

p_4_id2 = new_TripChain_2['버스노선ID2']
for i in p_4_id2:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_4'] += 1

p_4_id3 = new_TripChain_2['버스노선ID3']
for i in p_4_id3:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_4'] += 1

p_4_id4 = new_TripChain_2['버스노선ID4']
for i in p_4_id4:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_4'] += 1

p_4_id5 = new_TripChain_2['버스노선ID5']
for i in p_4_id5:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_4'] += 1

In [113]:
Route_Mapping

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung,p_1,p_2,p_4,p_8
53,경기시내,경기고속,4100200,234000068,41002090,720-3,True,17211,68,899,0
759,경기시내,경남여객,4100600,228000204,41006281,24,True,266,0,13,0
862,경기시내,경진여객,4100800,233000010,41008001,33-1,True,203,0,15,0
863,경기시내,경진여객,4100800,233000013,41008101,2,True,4,0,0,0
864,경기시내,경진여객,4100800,233000015,41008103,4,True,7,0,0,0
865,경기시내,경진여객,4100800,233000019,41008110,5-2,True,2,0,0,0
866,경기시내,경진여객,4100800,233000021,41008112,5-4,True,1,0,0,0
867,경기시내,경진여객,4100800,233000022,41008115,15,True,0,0,2,0
868,경기시내,경진여객,4100800,233000024,41008114,14,True,0,0,0,0
869,경기시내,경진여객,4100800,233000025,41008105,9,True,3,0,0,0


경로(8)의 버스노선ID1~5

In [114]:
new_TripChain_8 = TripChain[TripChain['사용자구분']==8]
new_TripChain_8.shape

(2615, 51)

In [115]:
p_8_id1 = new_TripChain_8['버스노선ID1']
for i in p_8_id1:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_8'] += 1

p_8_id2 = new_TripChain_2['버스노선ID2']
for i in p_8_id2:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_8'] += 1

p_8_id3 = new_TripChain_2['버스노선ID3']
for i in p_8_id3:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_8'] += 1

p_8_id4 = new_TripChain_2['버스노선ID4']
for i in p_8_id4:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_8'] += 1

p_8_id5 = new_TripChain_2['버스노선ID5']
for i in p_8_id5:
    if i in normal_id_h:
        Route_Mapping.loc[Route_Mapping['표준노선ID'] == i, 'p_8'] += 1
Route_Mapping

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung,p_1,p_2,p_4,p_8
53,경기시내,경기고속,4100200,234000068,41002090,720-3,True,17211,68,899,48
759,경기시내,경남여객,4100600,228000204,41006281,24,True,266,0,13,0
862,경기시내,경진여객,4100800,233000010,41008001,33-1,True,203,0,15,0
863,경기시내,경진여객,4100800,233000013,41008101,2,True,4,0,0,0
864,경기시내,경진여객,4100800,233000015,41008103,4,True,7,0,0,0
865,경기시내,경진여객,4100800,233000019,41008110,5-2,True,2,0,0,0
866,경기시내,경진여객,4100800,233000021,41008112,5-4,True,1,0,0,0
867,경기시내,경진여객,4100800,233000022,41008115,15,True,0,0,2,0
868,경기시내,경진여객,4100800,233000024,41008114,14,True,0,0,0,0
869,경기시내,경진여객,4100800,233000025,41008105,9,True,3,0,0,0


In [35]:
sor = Route_Mapping.sort_values(['p_1', 'p_4'], ascending=[False, False])

In [38]:
sor.to_csv('hwasung_routemapping.csv')

In [39]:
t = pd.read_csv('hwasung_routemapping.csv')

In [42]:
sorted_route_mapping[40:]

,Unnamed: 0,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung,p_1,p_2,p_4,p_8
40,894,경기시내,경진여객,4100800,233000058,41008140,7,True,9,0,3,0
41,907,경기시내,경진여객,4100800,233000095,41008156,26,True,8,0,0,0
42,864,경기시내,경진여객,4100800,233000015,41008103,4,True,7,0,0,0
43,870,경기시내,경진여객,4100800,233000026,41008117,19,True,6,0,0,0
44,2106,경기시내,(주)화성여객,4108600,233000142,41086010,6002-1,True,6,0,0,0
45,872,경기시내,경진여객,4100800,233000028,41008102,3,True,5,0,0,0
46,888,경기시내,경진여객,4100800,233000048,41008131,1,True,5,0,0,0
47,863,경기시내,경진여객,4100800,233000013,41008101,2,True,4,0,0,0
48,883,경기시내,경진여객,4100800,233000043,41008126,6-2,True,4,0,0,0
49,885,경기시내,경진여객,4100800,233000045,41008128,6-3,True,4,0,0,0


In [ ]:
# 시간별 
# 1. 7시에 사람들이 얼마나 타는지
# 2. 노선별 탑 10개
# 
time_시간별.ipynb

내가 할 일
```
1. folium으로 상위 5개 노선 한번에 그리고 저장
2. 전체적으로 화성버스를 타는 성인/어린이/청소년/노인 수 확인
```

In [45]:
TripChain['승차일시1'] = pd.to_datetime(TripChain['승차일시1'], format='%Y%m%d%H%M%S')
TripChain['승차일시2'] = pd.to_datetime(TripChain['승차일시2'], format='%Y%m%d%H%M%S')
TripChain['승차일시3'] = pd.to_datetime(TripChain['승차일시3'], format='%Y%m%d%H%M%S')
TripChain['승차일시4'] = pd.to_datetime(TripChain['승차일시4'], format='%Y%m%d%H%M%S')
TripChain['승차일시5'] = pd.to_datetime(TripChain['승차일시5'], format='%Y%m%d%H%M%S')
TripChain['하차일시1'] = pd.to_datetime(TripChain['하차일시1'], format='%Y%m%d%H%M%S')
TripChain['하차일시2'] = pd.to_datetime(TripChain['하차일시2'], format='%Y%m%d%H%M%S')
TripChain['하차일시3'] = pd.to_datetime(TripChain['하차일시3'], format='%Y%m%d%H%M%S')
TripChain['하차일시4'] = pd.to_datetime(TripChain['하차일시4'], format='%Y%m%d%H%M%S')
TripChain['하차일시5'] = pd.to_datetime(TripChain['하차일시5'], format='%Y%m%d%H%M%S')

In [52]:
day_hash = {'1': 0, '2': 0, '3':0, '4':0, 'nan':0}


for a in range(len(TripChain)):
    tmp = str(TripChain['승차일시1'][a].day)
    day_hash[tmp] += 1

In [53]:
day_hash

{'1': 181021, '2': 365018, '3': 404486, '4': 98016, 'nan': 34}

In [ ]:
2606개중 1123개가 없음!!(성보님이 보내주신 사진 이용)

In [8]:
RouteStationMapping.shape

NameError: name 'All_bus_id' is not defined